![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor

# import

In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
%matplotlib inline


# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [ ]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [ ]:
df_train.info()

In [ ]:
df_train.head(5)

In [ ]:
df_test.info()

In [ ]:
df_test.head(5)

In [ ]:
sample_submission.head(5)

In [ ]:
sample_submission.info()

In [ ]:
# ВАЖНО! для корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [ ]:
data.sample(5)

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

# Cleaning and Prepping Data

## Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

In [ ]:
data.info()

In [ ]:
def isnan_feature(data):
    # makes new feature with 1 if value in existing feature is NaN else 0
    for col in data.columns[data.isna().nunique() == 2]:
        data[col.replace(' ','_')+'_isNaN'] = pd.isna(data[col]).astype('uint8')
    return data


data = isnan_feature(data)

In [ ]:
data.info()

In [ ]:
# Далее заполняем пропуски в Number of Reviews нулем

def fill_zero(series):
    # fills NaN with 0
    series.fillna(0, inplace=True)


fill_zero(data['Number of Reviews'])

In [ ]:
data.info()

## Обработка признаков

In [ ]:
data.nunique(dropna=False)

### Restaurant_id:

In [ ]:
data.Restaurant_id.nunique()

Значения Restaurant_id не уникальны

In [ ]:
data.groupby('Restaurant_id').Restaurant_id.agg('count').sort_values()

In [ ]:
data[data.Restaurant_id == 'id_871'].City.value_counts()

In [ ]:
data[(data.Restaurant_id == 'id_871') & (data.City == 'Berlin')]

Значения Restaurant_id уникальны для каждого города в тренировочной выборке. Те же значения могут относится к другому ресторану в тестовой выборке. Для модели признак бесполезен.

### Price Range:

In [ ]:
data['Price Range'].value_counts()

По описанию 'Price Range' это - Цены в ресторане.  
Их можно поставить по возрастанию (значит это не категориальный признак). А это значит, что их можно заменить последовательными числами, например 1,2,3  

In [ ]:
def price_range_to_digits(series):
    # change values from '$','$$ - $$$','$$$$' to digits 1,2,3; fills NaN with mode (2)
    series = series.apply(lambda x: 1 if x=='$' else (3 if x=='$$$$' else 2))
    return series


data['Price Range'] = price_range_to_digits(data['Price Range'])

In [ ]:
data['Price Range'].isna().value_counts()

In [ ]:
data.info()

### City:

In [ ]:
data.City.unique()

In [ ]:
len(data.City.unique())

Признак City содержит названия крупных европейских городов. Добавим больше информации о городах из внешних источников.

In [ ]:
# внешний датасет, содержащий информацию о населении городов и их координатах:
world_cities = pd.read_csv('/kaggle/input/world-cities/worldcities.csv',
                           usecols=['city_ascii',
                                    'capital',
                                   'population'])

# приведение в соответствие с рабочим датасетом названия города Порто:
world_cities.city_ascii[world_cities.city_ascii == 'Porto'] = 'Oporto'
world_cities = world_cities[world_cities.city_ascii.isin(data.City)]
# предположительно, рестораны находятся в крупнейших городах из одноименных
world_cities.sort_values(['city_ascii','population'], inplace=True)
world_cities.drop_duplicates('city_ascii', keep='last', inplace=True)
world_cities.capital = world_cities.capital.apply(lambda x: 1 if x=='admin' else 0)
world_cities

In [ ]:
world_cities.shape

In [ ]:
world_cities.columns = ['City',
                        'capital',
                        'population']

In [ ]:
data = data.merge(world_cities, on='City')

In [ ]:
data

In [ ]:
data.info()

Добавим ещё больше информации о городах:

In [ ]:
qol = pd.read_csv('/kaggle/input/city-quality-of-life-dataset/uaScoresDataFrame.csv',
                  usecols=['UA_Name', 
#                            'UA_Country', 
#                            'UA_Continent', 
                           'Housing', 
                           'Cost of Living', 
#                            'Startups', 
#                            'Venture Capital', 
                           'Travel Connectivity',
                           'Commute', 
#                            'Business Freedom', 
                           'Safety', 
#                            'Healthcare', 
#                            'Education',
#                            'Environmental Quality',
                           'Economy',
#                            'Taxation',
#                            'Internet Access',
                           'Leisure & Culture', 
#                            'Tolerance', 
#                            'Outdoors'
                          ])

qol.UA_Name[qol.UA_Name == 'Porto'] = 'Oporto'
qol = qol[qol.UA_Name.isin(data.City)]

In [ ]:
qol

In [ ]:
qol.info()

In [ ]:
data = data.merge(qol, left_on='City', right_on='UA_Name', how='left')

In [ ]:
data.info()

In [ ]:
# для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
data = data.join(pd.get_dummies(data['City'], dummy_na=True))

In [ ]:
data.info()

### Cuisine Style:

In [ ]:
data['Cuisine Style'].value_counts()

In [ ]:
def cuisine_style(data):
    # генерация dummy признаков по видам кухни:
    data['Cuisine Style'] = data['Cuisine Style'].str.strip('[] ')
    cuisines = data['Cuisine Style'].str.get_dummies(sep=', ')
    cuisines.columns = cuisines.columns.str.strip("'")

    # создание отдельного столбца с количеством заявленных рестораном видов кухни:
    cuisines['number_of_cuisine_styles'] = cuisines.agg('sum', axis = 1)
    data = data.join(cuisines)
    return data


data = cuisine_style(data)

In [ ]:
data.number_of_cuisine_styles.value_counts()

### Reviews:

In [ ]:
data.Reviews

In [ ]:
type(data.Reviews[0])

Значения представляют собой вложенные списки в строковом формате

In [ ]:
import ast

In [ ]:
def reviews(data):
    data.Reviews.fillna('[[], []]', inplace=True)
    # ast.literal_eval не работает с 'nan'
    data.Reviews = data.Reviews.str.replace('nan,', "'',")
    data.Reviews = data.Reviews.str.replace(' nan', " ''")
    
    # преобразование строковых значений в список
    data.Reviews = data.Reviews.apply(lambda x: ast.literal_eval(x))
    
    # выделение текста каждого отзыва в отдельные колонки
    data['rev_text'] = data.Reviews.apply(lambda x: x[0])
    data['rev1_text'] = data.rev_text.apply(lambda x: x[0] if x != [] else None)
    data['rev2_text'] = data.rev_text.apply(lambda x: x[1] if len(x) == 2 else None)

    # выделение времени оставления каждого отзыва в отдельные колонки
    data['rev_date'] = data.Reviews.apply(lambda x: x[1])
    data['rev1_date'] = data.rev_date.apply(lambda x: pd.to_datetime(x[0]) if x != [] else None)
    data['rev2_date'] = data.rev_date.apply(lambda x: pd.to_datetime(x[1]) if len(x) == 2 else None)
    
    # Создание признака, содержащего количество дней между двумя отзывами:
    data['rev_delta'] = abs((data.rev1_date - data.rev2_date).dt.days)
    data.rev_delta.fillna(0, inplace=True)
    
    # Создание признаков, содержащих для каждого отзыва год, код дня недели, день в году:
    data['rev1_y'] = data.rev1_date.apply(lambda x: pd.Timestamp(x).year)
    data['rev2_y'] = data.rev2_date.apply(lambda x: pd.Timestamp(x).year)
    data['rev1_w'] = data.rev1_date.apply(lambda x: pd.Timestamp(x).weekday())
    data['rev2_w'] = data.rev2_date.apply(lambda x: pd.Timestamp(x).weekday())
    data['rev1_d'] = data.rev1_date.apply(lambda x: pd.Timestamp(x).dayofyear)
    data['rev2_d'] = data.rev2_date.apply(lambda x: pd.Timestamp(x).dayofyear)
    
    # Заполнение пропусков средним значением:
    
    data.rev1_y.fillna(data.rev1_y.median(), inplace=True)
    data.rev2_y.fillna(data.rev2_y.median(), inplace=True)

    data.rev1_w.fillna(data.rev1_w.median(), inplace=True)
    data.rev2_w.fillna(data.rev2_w.median(), inplace=True)

    data.rev1_d.fillna(data.rev1_d.median(), inplace=True)
    data.rev2_d.fillna(data.rev2_d.median(), inplace=True)
    
    # Создание признака, содержащего количество отзывов на сайте:
    data['number_of_reviews_site'] = data.rev_text.apply(lambda x: len(x))
    
    # Создание признаков, содержащих длину каждого отзыва и среднюю длину двух отзывов, заполнение пропусков нулями:
    data['rev1_len'] = data.rev1_text.str.len()
    data['rev2_len'] = data.rev2_text.str.len()

    data[['rev1_len', 'rev2_len']] = data[['rev1_len', 'rev2_len']].fillna(0)

    data['rev_len_mean'] = (data.rev1_len + data.rev2_len)/2
    
    
    data.drop(['rev_text', 'rev_date', 'rev1_text', 'rev2_text', 'rev1_date', 'rev2_date'], axis=1, inplace=True)
    
    
    
    
    return data

In [ ]:
data = reviews(data)

### Создание нового признака с количеством ресторанов в городе:

In [ ]:
Restaurants_in_City = data.City.value_counts()

In [ ]:
data['Restaurants_in_City'] = data.City.apply(lambda x: Restaurants_in_City[x])

In [ ]:
def restaurants_in_city(data):
    Restaurants_in_City = data.City.value_counts()
    data['Restaurants_in_City'] = data.City.apply(lambda x: Restaurants_in_City[x])
    return data

In [ ]:
data.info()

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [ ]:
# на всякий случай, заново подгружаем данные
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()

In [ ]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    df_output.drop(['Restaurant_id','ID_TA',], axis = 1, inplace=True)
    
    
    # ################### 2. NAN ############################################################## 
    # Далее заполняем пропуски
    df_output = isnan_feature(df_output)
    df_output.drop(['Number_of_Reviews_isNaN'], axis=1, inplace=True)

    fill_zero(df_output['Number of Reviews'])
    
    
    # ################### 3. Encoding & Feature Engineering ############################################################## 
    
    df_output['Price Range'] = price_range_to_digits(df_output['Price Range'])

    df_output = df_output.join(pd.get_dummies(df_output.City, dummy_na=True))
    
    df_output = df_output.merge(world_cities, on='City', how='left')
    
    df_output = df_output.merge(qol, left_on='City', right_on='UA_Name', how='left')
    
    # признак снижает значение метрики
    df_output.drop(['Cuisine_Style_isNaN'], axis=1, inplace=True)
    
    df_output = cuisine_style(df_output)
     
    df_output = restaurants_in_city(df_output)
    
    df_output = reviews(df_output)
    
    
    # ################### 4. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    
    return df_output

#### Запускаем и проверяем что получилось

In [ ]:
df_preproc = preproc_data(data)
df_preproc.sample(10)

In [ ]:
df_preproc.info()

In [ ]:
df_preproc.isna().agg('sum').index.to_list()

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

Значения рейтинга представлены с шагом 0,5. Округлим результат работы модели:

In [ ]:
y_pred = (y_pred*2).round()/2
y_pred

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# MAE: 0.21240125

In [ ]:
# MAE: 0.21087374999999997
# isNaN

In [ ]:
# MAE: 0.210635
# price range

In [ ]:
# MAE: 0.209500625
# все кухни в дамми

In [ ]:
# MAE: 0.209245
# убрана колонка Cuisine_Style_isNaN

In [ ]:
# MAE: 0.20915249999999996
# убрана колонка Number_of_Reviews_isNaN

In [ ]:
# MAE: 0.207923125
#  добавлен world_cities без координат

In [ ]:
# MAE: 0.20692687499999998
# добавлен qol

In [ ]:
# MAE: 0.180375
# округление рейтинга

In [ ]:
# MAE: 0.1794375
# restaurants_in_city

In [ ]:
# MAE: 0.1778125
# reviews

In [ ]:
# MAE: 0.1736875
#     reviews2

In [ ]:
# MAE: 0.1723125
# reviews3

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['Rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
# округление рейтинга до 0.5
predict_submission = (predict_submission * 2).round() / 2
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)